# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
# Install pyspark
!pip install pyspark
# import findspark
# findspark.init()
import os
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
You are working with 1 core(s)


In [3]:
path = '/content/drive/MyDrive/Data Science Intake43/5. Spark/spark-scripts/section2/Datasets/'
os.listdir(path)

['people.json',
 'zomato.csv',
 'youtubevideos.csv',
 'googleplaystore.csv',
 'fifa19.csv',
 'students.csv',
 'users1.parquet',
 'users3.parquet',
 'users2.parquet',
 'nyc_air_bnb.csv',
 'rec-crime-pfa.csv',
 'supermarket_sales.csv',
 'Rep_vs_Dem_tweets.csv',
 'pga_tour_historical.csv',
 'uw-madison-courses']

## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [4]:
df = spark.read.csv(path+'googleplaystore.csv',inferSchema=True,header=True)

In [5]:
# data shape
print((df.count(), len(df.columns)))

(10841, 13)


## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [6]:
df.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


As well as the schema to make sure all the column types were correctly infered

In [7]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [8]:
from pyspark.sql.types import IntegerType, FloatType
newdf = df.withColumn("Rating", df["Rating"].cast(FloatType())) \
            .withColumn("Reviews", df["Reviews"].cast(IntegerType())) \
            .withColumn("Price", df["Price"].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [9]:
# Create a temporary view of the dataframe
df.createOrReplaceTempView("tempview")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [10]:
spark.sql("SELECT * FROM tempview WHERE Rating > 4.1").limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
3,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
4,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [11]:
new_df = spark.sql("SELECT app, rating, category FROM tempview WHERE category = 'COMICS' and Rating > 4.5")
new_df.limit(5).toPandas()

,app,rating,category
0,Manga Master - Best manga & comic reader,4.6,COMICS
1,GANMA! - All original stories free of charge f...,4.7,COMICS
2,Röhrich Werner Soundboard,4.7,COMICS
3,Unicorn Pokez - Color By Number,4.8,COMICS
4,Manga - read Thai translation,4.6,COMICS


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [12]:
spark.sql("SELECT category, sum(reviews) total_reviews FROM tempview group by category order by total_reviews desc").limit(1).toPandas()

,category,total_reviews
0,GAME,1.585422e+09


## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [13]:
spark.sql('select app, sum(reviews) total_reviews from tempview group by app order by total_reviews desc').show(1)

+---------+-------------+
|      app|total_reviews|
+---------+-------------+
|Instagram| 2.66241989E8|
+---------+-------------+
only showing top 1 row



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [14]:
spark.sql('select app from tempview where lower(app) like "%dating%" ').show(10, False)

+--------------------------------------------------+
|app                                               |
+--------------------------------------------------+
|Private Dating, Hide App- Blue for PrivacyHider   |
|AMBW Dating App: Asian Men Black Women Interracial|
|Zoosk Dating App: Meet Singles                    |
|OkCupid Dating                                    |
|Match™ Dating - Meet Singles                      |
|Hily: Dating, Chat, Match, Meet & Hook up         |
|Hinge: Dating & Relationships                     |
|Casual Dating & Adult Singles - Joyride           |
|BBW Dating & Plus Size Chat                       |
|CMB Free Dating App                               |
+--------------------------------------------------+
only showing top 10 rows



## 6. Use SQL Transformer to display how many free apps there are in this list

In [15]:
# First we need to import SQL transformer
from pyspark.ml.feature import SQLTransformer

In [16]:
# Then we create an SQL call 
sqlTrans = SQLTransformer(statement='select count(*) free_apps_count from tempview where type = "Free"') 
sqlTrans.transform(df).show()

+---------------+
|free_apps_count|
+---------------+
|          10037|
+---------------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [17]:
spark.sql('select genres, count(*) genre_appearences from tempview group by genres order by genre_appearences desc').show(1)

+------+-----------------+
|genres|genre_appearences|
+------+-----------------+
| Tools|              842|
+------+-----------------+
only showing top 1 row



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [18]:
spark.sql('select app, genres, reviews from tempview where genres = "Tools" and reviews>100 ').show(10)

+--------------------+------+--------+
|                 app|genres| reviews|
+--------------------+------+--------+
|   Moto File Manager| Tools|   38655|
|              Google| Tools| 8033493|
|    Google Translate| Tools| 5745093|
|        Moto Display| Tools|   18239|
|      Motorola Alert| Tools|   24199|
|     Motorola Assist| Tools|   37333|
|Cache Cleaner-DU ...| Tools|12759663|
|  Moto Suggestions ™| Tools|     308|
|          Moto Voice| Tools|   33216|
|          Calculator| Tools|   40770|
+--------------------+------+--------+
only showing top 10 rows



## That's all folks! Great job!